In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook

In [3]:
mtx = np.array([[9.842439e+02,0,6.900000e+02],
                [0,9.808141e+02,2.331966e+02],
                [0,0,1]])

### Initialize odometry

In [93]:
def load_images(dataset='straight'):
    if dataset == 'straight':
        imgs = glob.glob('./data/straight/*.png')
    elif dataset == 'bicycle':
        imgs = glob.glob('./data/bicycle/*.png')
    elif dataset == 'highway':
        imgs = glob.glob('./data/highway/*.png')
    elif dataset == 'lidl':
        imgs = glob.glob('./data/lidl/*.png')
    else:
        raise Exception('Invalid dataset')
    assert imgs, 'Couldnt locate images'
    print(f'Images: {len(imgs)}')
    return imgs

In [97]:
import sys
print(sys.version)

3.8.9 (tags/v3.8.9:a743f81, Apr  6 2021, 14:02:34) [MSC v.1928 64 bit (AMD64)]


In [157]:
imgs = load_images(dataset='bicycle')

# prepare live plot
output_notebook()

position = dict(x=[0], y=[0])
position2 = dict(x=[0], y=[0])
pos_data = ColumnDataSource(data=dict(x=[0], y=[0]))
pos_data2 = ColumnDataSource(data=dict(x=[0], y=[0]))

pos_figure = figure(plot_width=800, plot_height=400, x_range=(-400, 400), y_range=(-100, 800))
pos_figure.scatter("x", "y", source=pos_data)
pos_figure.scatter("x", "y", source=pos_data2, color='red')
handle = show(pos_figure, notebook_handle=True)


P_old = np.zeros((3,1))
R_old = np.eye(3)
t_old = np.ones(3)[:,np.newaxis]
t_old2 = t_old
try:
    for idx, fname in enumerate(imgs):
        img_new = cv2.imread(fname, 0)
        if idx == 0: #Reading first frame
            cv2.imshow("Image", img_new)
            cv2.waitKey(1)
            img_old = img_new
            continue

        F, pts1, pts2 = matchFeatures(img_new, img_old, feature_ratio=0.8)
        P, R, t, relative_scale = updatePos(P_old, F, pts1, pts2, mtx)
        
        # vizualize positions
        position['x'] = t_old[0]
        position['y'] = t_old[2]
        position2['x'] = t_old2[0]
        position2['y'] = t_old2[2]
        pos_data.stream(position, len(imgs))
        pos_data2.stream(position2, len(imgs))
        push_notebook(handle=handle)
        # visualize epilines
        # try:
        #     display_epilines(F, pts1, pts2, img_new, img_old)
        # except Exception as e:
        #     pass
        # visualize camera
        cv2.imshow("Image", img_new)
        cv2.waitKey(1)
        
        # https://github.com/uoip/monoVO-python/blob/master/visual_odometry.py#L83
        img_old = img_new
        t_old = t_old + relative_scale * R_old@t
        t_old2 = t_old2 + R_old@t
        R_old = R@R_old
        P_old = P
except Exception as e:
    cv2.destroyAllWindows()
    raise e

Images: 154


Loading BokehJS ...

Notes about algorithm

- Sensitive to significant turns
- Sometimes requires reruns
- When vehicle stops at cross-section and other cars are moving the position of camera changes
- Investigate changes in feature_ratio
- Relative scale doesnt change results

### Functions

In [148]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

def display_epilines(F, pts1, pts2, img1, img2):
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2, F)
    lines1 = lines1.reshape(-1,3)
    img5,img6 = drawlines(img1, img2, lines1, pts1, pts2)
    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1, F)
    lines2 = lines2.reshape(-1,3)
    img3,img4 = drawlines(img2, img1, lines2, pts2, pts1)
    cv2.imshow("Features",np.vstack((img5,img3)))
    cv2.waitKey(1)

def matchFeatures(img1, img2, feature_ratio=0.3):
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m, n) in enumerate(matches):
        if m.distance < feature_ratio * n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    return F, pts1, pts2
    
def updatePos(P, F, pts1, pts2, K):
    E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.8, threshold=1.0)
    
    num = np.sqrt(pts1[:,0] ** 2 + pts2[:,0] ** 2)
    den = np.sqrt(pts1[:,1] ** 2 + pts2[:,1] ** 2)
    relative_scale = np.median(num / den)
    
    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)
#     t = t + relative_scale * R.dot(t) 
#     P = relative_scale * R@P + t
    P = R@P + t
    return P, R, t, relative_scale

### Capture new frame

In [42]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(P[0,:], P[2,:], c='r', marker='o')
plt.show()
#plt.xlim([-25, 25])
plt.xlim([-np.max(P[2,:])/2, np.max(P[2,:])/2])

(-66.44053292359396, 66.44053292359396)